In [1]:
import glob
import time
import numpy as np
import pandas as pd
from geopy import distance
import datetime as datetime
from multiprocessing import Process
from sklearn.preprocessing import MinMaxScaler



In [2]:
def create_datetime_features(input_df, column_name):

    input_df[column_name+'_hour'] = input_df[column_name].dt.hour
    input_df[column_name+'_minute'] = input_df[column_name].dt.minute
    input_df[column_name+'_quarter'] = input_df[column_name].dt.quarter
    input_df[column_name+'_month'] = input_df[column_name].dt.month
    input_df[column_name+'_year'] = input_df[column_name].dt.year
    input_df[column_name+'_week'] = input_df[column_name].dt.isocalendar().week
    input_df[column_name+'_day'] = input_df[column_name].dt.day
    input_df[column_name+'_dayofweek'] = input_df[column_name].dt.dayofweek

    return input_df

In [3]:
def calculate_geodesic_distance(df_stations, point):
    df = df_stations.copy()
    for idx, i, j  in zip(df.index, df.station_latitude, df.station_longitude):
        df.loc[idx, 'distance'] = distance.distance(point, (i,j)).m
        if int(df.loc[idx, 'distance']) == 0:
            
            return df.loc[idx, "station_name"], df.loc[idx, 'distance']
    df = df.sort_values("distance")
    return df.iloc[0]["station_name"], df.iloc[0]["distance"]

In [4]:
def get_approximate_stations_locations(df_missing_stations, start_stations, save_file, col ="start_station"):
    count = 0
    for idx, i, j in zip(df_missing_stations.index, 
                         df_missing_stations[f'{col}_latitude'], 
                         df_missing_stations[f'{col}_longitude']):
        
        approx_station, approx_distance = calculate_geodesic_distance(start_stations, 
                            (i, j))
        df_missing_stations.loc[idx,f"{col}_approx"] = approx_station
        df_missing_stations.loc[idx,f"{col}_approx_distance"] = approx_distance
        
        if count%1000 == 0:
            print(count)
            df_missing_stations.to_csv(f"{save_file}")
        count += 1
    df_missing_stations.to_csv(f"{save_file}")
    return df_missing_stations

In [5]:
def parallel_get_approximate_stations_locations(df_missing_stations, start_stations, 
                            save_file_suffix="approx_stations", batch_size=100000, sleep_time=120):

    ranges = np.arange(0, df_missing_stations.shape[0], 
                       batch_size, dtype=int)
    ranges = np.concatenate((ranges, [df_missing_stations.shape[0]]))
    x = 0
    processes = []
    for ii in range(1, len(ranges)):
        print(ranges[ii-1], ranges[ii])
        save_file = f"approximate_stations/{save_file_suffix}_{ii}.csv"
        partitioned_df = df_missing_stations.loc[ranges[ii-1]: 
                                    ranges[ii]].copy().reset_index(drop=True)    
        
        p = Process(target=get_approximate_stations_locations, 
                            args = (partitioned_df, start_stations, save_file))
        #get_approximate_stations_locations(partitioned_df, start_stations, save_file)
        x +=1
        p.start()
        print(x)
        processes.append(p)
        time.sleep(sleep_time)
    for thread in processes:
        thread.join()

# Importing and renaming columns

In [6]:
df_2021 = pd.DataFrame()
df_2022 = pd.DataFrame()
df_initial = pd.DataFrame()

files_2021 = [i for i in glob.glob("*.csv") if "2021" in i]
files_2022 = [i for i in glob.glob("*.csv") if "2022" in i]

for file in files_2021:
    df_2021 = df_2021.append(pd.read_csv(file).rename(columns={
        "started_at": "start_time",
        "ended_at": "end_time",
        "start_lat": "start_station_latitude",
        "start_lng": "start_station_longitude",
        "end_lat": "end_station_latitude",
        "end_lng": "end_station_longitude",
        "member_casual": "user_type"
    }))
for file in files_2022:
    df_2022 = df_2022.append(pd.read_csv(file).rename(columns={
        "started_at": "start_time",
        "ended_at": "end_time",
        "start_lat": "start_station_latitude",
        "start_lng": "start_station_longitude",
        "end_lat": "end_station_latitude",
        "end_lng": "end_station_longitude",
        "member_casual": "user_type"
    }))   
    
df_initial = df_2021.append(df_2022).reset_index(drop=True)

In [7]:
df_initial.shape

(4513199, 13)

In [8]:
del df_2021
del df_2022

In [9]:
df_initial = df_initial.dropna(subset=[
    'start_station_latitude',
    'start_station_longitude',
    'end_station_latitude',
    'end_station_longitude'])

In [10]:
df_initial['start_time'] = pd.to_datetime(df_initial['start_time'])

In [11]:
df_initial = df_initial[df_initial['start_time'] >= datetime.datetime(2021, 1, 1)]

# Finding Standard Stations

In [12]:
start_stations = df_initial[["start_station_name", 
                             'start_station_latitude', 
                             "start_station_longitude"]]\
.sort_values(['start_station_latitude', "start_station_longitude"])\
.drop_duplicates(subset=['start_station_latitude','start_station_longitude'])\
.dropna().drop_duplicates(subset=['start_station_name'])\
.query("start_station_latitude != 0").reset_index(drop=True).rename(
    columns={"start_station_name": "station_name", 
            'start_station_latitude': "station_latitude",
            "start_station_longitude": "station_longitude"})

In [13]:
end_stations = df_initial[["end_station_name", 
                           'end_station_latitude', 
                           "end_station_longitude"]]\
.sort_values(['end_station_latitude', "end_station_longitude"])\
.drop_duplicates(subset=['end_station_latitude','end_station_longitude'])\
.dropna().drop_duplicates(subset=['end_station_name'])\
.query("end_station_latitude != 0").reset_index(drop=True).rename(
    columns={"end_station_name": "station_name", 
            'end_station_latitude': "station_latitude",
            "end_station_longitude": "station_longitude"})

In [14]:
standard_stations = start_stations.append(end_stations).sort_values(
    ['station_latitude', "station_longitude"]).drop_duplicates(subset=['station_name'])

# Trip duration calculation

In [15]:
df_initial['start_time'] = pd.to_datetime(df_initial['start_time'])
df_initial['end_time'] = pd.to_datetime(df_initial['end_time'])
df_initial['duration_sec'] = (df_initial['end_time'] - df_initial['start_time'])/ pd.Timedelta(seconds=1)


df_initial['user_type'] = df_initial['user_type'].replace({"member":"Subscriber", "casual": "Customer"})

Removing 2020 data for irrelevancy 

# Lookup missing Station Names

Approximating missing stations

In [16]:
df_missing_stations = df_initial[(df_initial['start_station_name'].isna()) |\
                                (df_initial['end_station_name'].isna())].copy()

df_existing_stations = df_initial[~((df_initial['start_station_name'].isna()) |\
                                    (df_initial['end_station_name'].isna()))].copy()


In [17]:
del df_initial

In [18]:
df_missing_start_stations_no_duplicates = df_missing_stations.drop_duplicates(
                                        subset=['start_station_latitude', 'start_station_longitude']).copy()\
                                            .reset_index(drop=True)
df_missing_start_stations_no_duplicates = df_missing_start_stations_no_duplicates[
                                            df_missing_start_stations_no_duplicates['start_station_name'].isna()].copy()\
                                            .reset_index(drop=True)

In [19]:
df_missing_end_stations_no_duplicates = df_missing_stations.drop_duplicates(
                                        subset=['end_station_latitude', 'end_station_longitude']).copy()\
                                            .reset_index(drop=True)
df_missing_end_stations_no_duplicates = df_missing_end_stations_no_duplicates[
                                            df_missing_end_stations_no_duplicates['end_station_name'].isna()].copy()\
                                                .reset_index(drop=True)

In [20]:
approximate_start_stations_df = get_approximate_stations_locations(df_missing_start_stations_no_duplicates, 
                                   standard_stations.reset_index(drop=True).copy(), 
                                        save_file="approximate_stations/approx_start_stations.csv", 
                                            col ="start_station")

0


In [21]:
approximate_end_stations_df = get_approximate_stations_locations(df_missing_end_stations_no_duplicates, 
                                   standard_stations.reset_index(drop=True).copy(), 
                                     save_file="approximate_stations/approx_end_stations.csv", 
                                         col ="end_station")

0


In [22]:
approximate_start_stations_df = pd.read_csv("approximate_stations/approx_start_stations.csv") 
approximate_end_stations_df = pd.read_csv("approximate_stations/approx_end_stations.csv") 

In [23]:
df_missing_stations = df_missing_stations.merge(
    approximate_start_stations_df[["start_station_approx", 
                             'start_station_latitude',
                             'start_station_longitude',
                             'start_station_approx_distance']],
    on=['start_station_latitude', 
        'start_station_longitude'], how="left")

In [24]:
df_missing_stations = df_missing_stations.merge(
    approximate_end_stations_df[["end_station_approx", 
                             'end_station_latitude',
                             'end_station_longitude',
                             'end_station_approx_distance']],
    on=['end_station_latitude', 
        'end_station_longitude'], how="left")

In [25]:
index = df_missing_stations['start_station_approx_distance'] < 500
df_missing_stations.loc[index, 'start_station_name'] = df_missing_stations.loc[index, 'start_station_approx']

index = df_missing_stations['end_station_approx_distance'] < 500
df_missing_stations.loc[index, 'end_station_name'] = df_missing_stations.loc[index, 'end_station_approx']

In [26]:
df_completed_stations = df_missing_stations.dropna(subset=['start_station_name', 'end_station_name'])

In [27]:
df_existing_stations = df_existing_stations.drop(columns=['start_station_id', 'end_station_id'])
df_completed_stations = df_completed_stations.drop(columns=['start_station_id', 'end_station_id'])

In [28]:
df_existing_stations

,ride_id,rideable_type,start_time,end_time,start_station_name,end_station_name,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,user_type,duration_sec
3236,ABDA3A49559A3FAE,classic_bike,2021-01-07 22:15:32,2021-01-07 22:31:47,Vine St at Shattuck Ave,Vine St at Shattuck Ave,37.880222,-122.269592,37.880222,-122.269592,Customer,975.0
6110,589DBD2D0F0902D0,classic_bike,2021-01-18 15:06:21,2021-01-18 16:39:39,Vine St at Shattuck Ave,Vine St at Shattuck Ave,37.880222,-122.269592,37.880222,-122.269592,Customer,5598.0
6232,7B7344B07AC7ACC1,classic_bike,2021-01-28 18:07:00,2021-01-28 18:16:54,Carl St at Cole St,Haight St at Lyon St,37.765942,-122.449228,37.770519,-122.442326,Subscriber,594.0
6233,99E98580EAB8D099,electric_bike,2021-01-03 15:03:16,2021-01-03 15:14:38,Carl St at Cole St,Grove St at Divisadero,37.765924,-122.449284,37.775969,-122.437662,Customer,682.0
6234,4A477E3D8717E550,electric_bike,2021-01-25 11:19:22,2021-01-25 11:22:40,Carl St at Cole St,Haight St at Lyon St,37.765948,-122.449321,37.770540,-122.442437,Subscriber,198.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4513076,2887E37A53BF8674,classic_bike,2022-11-03 11:16:19,2022-11-03 11:31:33,Buchanan St at North Point St,Lincoln Blvd at Graham St,37.804433,-122.433523,37.801750,-122.456363,Customer,914.0
4513102,CA2ABE4595B3484A,classic_bike,2022-11-09 08:26:03,2022-11-09 08:33:35,14th St at Filbert St,West Oakland BART Station,37.808750,-122.283282,37.805190,-122.294617,Subscriber,452.0
4513131,6BA38879B20BFA13,classic_bike,2022-11-10 10:19:27,2022-11-10 10:26:45,14th St at Filbert St,West Oakland BART Station,37.808750,-122.283282,37.805190,-122.294617,Customer,438.0
4513149,15C5F50B2211BD4A,classic_bike,2022-11-01 18:28:22,2022-11-01 19:08:37,Bancroft Way at Telegraph Ave,West Oakland BART Station,37.868911,-122.258786,37.805190,-122.294617,Subscriber,2415.0


In [29]:
df = df_completed_stations.append(df_existing_stations).copy()

In [30]:
del df_completed_stations
del df_existing_stations
del df_missing_stations

In [31]:
df.to_csv("baywheels_confirmed_stations.csv")

In [32]:
df.shape

(4069749, 16)

# Dropping Trips < X minutes duration, where start = end

X = 4

In [33]:
minutes = 4

In [34]:
df_same_station = df[(df['start_station_name'] == df['end_station_name']) & 
                     (df['duration_sec'] > minutes*60)]
df_no_same_station = df[(df['start_station_name'] != df['end_station_name'])]

In [35]:
df_final = df_no_same_station.append(df_same_station).reset_index(drop=True)#

In [36]:
df_final = create_datetime_features(df_final, "start_time")
df_final = create_datetime_features(df_final, "end_time")

In [37]:
df_final

,ride_id,rideable_type,start_time,end_time,start_station_name,end_station_name,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,...,start_time_day,start_time_dayofweek,end_time_hour,end_time_minute,end_time_quarter,end_time_month,end_time_year,end_time_week,end_time_day,end_time_dayofweek
0,357CDE244D24405B,electric_bike,2021-01-26 11:32:59,2021-01-26 11:38:21,3rd St at Townsend St,Parker Ave at McAllister St,37.760000,-122.410000,37.760000,-122.420000,...,26,1,11,38,1,1,2021,4,26,1
1,19A3E1F4211D0EE8,electric_bike,2021-01-26 14:16:37,2021-01-26 14:19:24,Powell St at Columbus Ave,3rd St at Townsend St,37.770000,-122.410000,37.760000,-122.410000,...,26,1,14,19,1,1,2021,4,26,1
2,27004D90ADC81AFF,electric_bike,2021-01-26 14:02:37,2021-01-26 14:06:35,Parker Ave at McAllister St,Powell St at Columbus Ave,37.760000,-122.420000,37.770000,-122.410000,...,26,1,14,6,1,1,2021,4,26,1
3,2F81FCA3D9CD056A,electric_bike,2021-01-26 15:03:05,2021-01-26 15:07:25,3rd St at Townsend St,Parker Ave at McAllister St,37.760000,-122.410000,37.760000,-122.420000,...,26,1,15,7,1,1,2021,4,26,1
4,B4F4680078748D61,electric_bike,2021-01-21 14:38:42,2021-01-21 14:43:26,3rd St at Townsend St,Parker Ave at McAllister St,37.760000,-122.410000,37.760000,-122.420000,...,21,3,14,43,1,1,2021,3,21,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3921359,2253EEA330D0F940,classic_bike,2022-11-23 14:24:35,2022-11-23 15:15:37,Lincoln Blvd at Graham St,Lincoln Blvd at Graham St,37.801750,-122.456363,37.801750,-122.456363,...,23,2,15,15,4,11,2022,47,23,2
3921360,C103E092DF2FD974,electric_bike,2022-11-23 14:26:44,2022-11-23 15:18:20,Lincoln Blvd at Graham St,Lincoln Blvd at Graham St,37.801799,-122.456323,37.801750,-122.456363,...,23,2,15,18,4,11,2022,47,23,2
3921361,5469394432504DFB,electric_bike,2022-11-23 14:27:20,2022-11-23 15:17:23,Lincoln Blvd at Graham St,Lincoln Blvd at Graham St,37.801809,-122.456404,37.801750,-122.456363,...,23,2,15,17,4,11,2022,47,23,2
3921362,7FB5B1DAB7A58B09,classic_bike,2022-11-04 11:34:25,2022-11-04 13:33:41,Lincoln Blvd at Graham St,Lincoln Blvd at Graham St,37.801750,-122.456363,37.801750,-122.456363,...,4,4,13,33,4,11,2022,44,4,4


In [38]:
df_final.to_csv("baywheels_cleaned.csv")


In [39]:
standard_stations.to_csv("standard_stations.csv")

In [40]:
standard_stations

,station_name,station_latitude,station_longitude
0,Bestor Art Park,37.278118,-121.825770
1,Locust St at Grant St,37.302419,-121.868460
2,Willow St at Blewett Ave,37.308742,-121.900190
3,Bird Ave at Coe Ave,37.310122,-121.894399
4,Park Ave at Laurel Grove Ln,37.310755,-121.925801
...,...,...,...
529,North Berkeley BART Station,37.873558,-122.283093
530,Shattuck Ave at Hearst Ave,37.873676,-122.268487
531,Hearst Ave at Euclid Ave,37.875112,-122.260553
532,Virginia St at Shattuck Ave,37.876573,-122.269528


In [41]:
#count = 0
#for idx, i, j in zip(df_missing_stations.index, 
#                     df_missing_stations['start_station_latitude'], 
#                     df_missing_stations['start_station_longitude']):
#    approx_station, approx_distance = calculate_geodesic_distance(start_stations, 
#                        (i, j))
#    df_missing_stations.loc[idx,"start_station_approx"] = approx_station
#    df_missing_stations.loc[idx,"start_station_approx_distance"] = approx_distance
#    count += 1
#    if count%10000 == 0:
#        print(count)
#        df_missing_stations.to_csv("missing_stations.csv")

In [42]:
#
#import plotly.express as px
#fig = px.density_mapbox(df_initial.head(100000), lat='start_station_latitude', lon='start_station_longitude', radius=2,
#                        center=dict(lat=0, lon=180), zoom=0,
#                        mapbox_style="stamen-terrain")
#fig.update_geos(fitbounds="locations")
#fig.show()

In [43]:
#from sklearn.metrics.pairwise import haversine_distances
#
#points_in_radians = df_initial[['start_station_latitude','start_station_longitude']].head(10000).apply(np.radians).values
#distances_in_km = haversine_distances(points_in_radians) * 6371

In [44]:
#distance_matrix = distances_in_km

#clustering = DBSCAN(min_samples=2)
#scaler = MinMaxScaler()
#df = df_initial.copy()
#df[['start_station_latitude', 'start_station_longitude', 
#    'end_station_latitude', 'end_station_longitude']] = \
#                            scaler.fit_transform(df[['start_station_latitude', 'start_station_longitude', 
#                                                     'end_station_latitude', 'end_station_longitude']])
#clusters = clustering.fit_predict(df[['start_station_latitude', 'start_station_longitude']].head(10000))